# Part3.1 线性回归模型实现

#### 3.1.1 生成数据集

In [32]:
# 导包
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [33]:
# 模型的真实参数
true_w = torch.tensor([2, -3.4])
true_b = 4.2
featurse, lables = d2l.synthetic_data(true_w, true_b, 1000)

#### 3.1.2 读取数据集

In [34]:
# 使用框架中现有的API来读取数据
def load_array(data_arrays, batch_size, is_train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [35]:
batch_size = 10
data_iter = load_array((featurse, lables), batch_size)
data_iter

In [36]:
# 要读取数据 需要构造iter构造Python迭代器，并使用next从迭代器中获取
next(iter(data_iter))

[tensor([[ 0.0648,  0.2539],
         [ 1.1548, -1.0863],
         [ 1.1892, -1.6117],
         [ 1.9249, -1.1755],
         [-0.6474, -1.0942],
         [ 0.7170,  1.5887],
         [-1.1358, -0.4238],
         [-0.8668, -0.0415],
         [ 0.7965, -1.3915],
         [-0.3698,  0.2504]]),
 tensor([[ 3.4611],
         [10.2035],
         [12.0664],
         [12.0474],
         [ 6.6294],
         [ 0.2290],
         [ 3.3742],
         [ 2.5923],
         [10.5213],
         [ 2.6167]])]

#### 3.1.3 定义模型

In [37]:
# nn是神经网络的缩写
from torch import nn

"""
    定义一个net变量, 它是一个Sequential类的实例
    它能将多层串联在一起。
    在Pytorch中,全连接层在Linear类中定义,我们将两个参数传递到
    nn.Linear中。第一个指定输入特征的形状,第二个指定输出特征形状。
    如上创建的数据集,第0层维度为2, 输出层维度为1。
"""
net = nn.Sequential(nn.Linear(2, 1))

#### 3.1.4 初始化模型参数

In [38]:
# 初始化模型参数
"""
    通过访问net[0]选择网络中的第一个图层
"""
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

#### 3.1.5 定义损失函数

In [39]:
# 定义损失函数

"""
    计算均方差使用的是MSELoss类,也称为L2范数.
"""
loss = nn.MSELoss()

#### 3.1.6 定义优化函数

In [40]:
# 定义优化算法

"""
    Pytorch在optim模块中实现了改算法的许多变种
    当实例化一个SGD实例时,指定优化参数(net.parameters())
    以及优化算法所需提供的超参数字典.
    mini-batch梯度算法只需要设置lr值
"""
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

#### 3.1.7 训练

In [41]:
# 设置迭代次数
num_epochs = 10

# 外循环遍历迭代次数
for epoch in range(num_epochs):
    # 内循环mini-batch
    for X, y in data_iter:
        # 计算损失函数
        l = loss(net(X), y)
        # 清空梯度缓存，否则后续会累加
        trainer.zero_grad()
        # 反向传播，求梯度
        l.backward()
        # 通过调用优化器更新参数
        trainer.step()
    l = loss(net(featurse), lables)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000258
epoch 2, loss 0.000099
epoch 3, loss 0.000099
epoch 4, loss 0.000098
epoch 5, loss 0.000099
epoch 6, loss 0.000098
epoch 7, loss 0.000099
epoch 8, loss 0.000098
epoch 9, loss 0.000099
epoch 10, loss 0.000098


In [42]:
w = net[0].weight.data
# reshape 防止维度不一致
print('W的估计误差:', true_w - w.reshape(true_w.shape))

b = net[0].bias.data
print('b的估计误差:',  true_b - b)

W的估计误差: tensor([8.7500e-05, 5.3072e-04])
b的估计误差: tensor([0.0003])
